In [1]:
# import requests
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from lxml import etree, html

In [2]:
browser = webdriver.Chrome("./chromedriver")
url = "https://www.ntuh.gov.tw/labmed/檢驗目錄/Lists/2015/BC.aspx"
tests = dict(test_ename = [], test_cname = [], specimen = [], reference_range = [], detail_link = [])

# Make the browser visit the page
browser.get(url)
# Attempt to turn the page 10 times
for i in range(10):
    # Transform what browser gets into an element tree
    root = etree.fromstring(browser.page_source, etree.HTMLParser())
    # Parse the information with Xpath
    for row in root.xpath("//table[@class='ms-listviewtable']/tbody/tr"):
        tests["test_ename"].append(row.xpath("./td/div/a/text()")[0].strip())
        tests["test_cname"].append(row.xpath("./td/text()")[0].strip())
        try:
            tests["specimen"].append(row.xpath("./td/div/font/text()")[0].strip())
        except IndexError:
            tests["specimen"].append(row.xpath("./td/div/div/text()")[0].strip())
        tests["reference_range"].append("\n".join(map(lambda x: x.strip(),row.xpath("./td/div/font/text()")[1:])))
        tests["detail_link"].append(row.xpath("./td/div/a/@href")[0].strip())
    try:
        browser.find_element_by_xpath("//td[@id='pagingWPQ2next']").click()
        sleep(5)
    except NoSuchElementException:
        # break the for loop if there is no next page
        break
browser.quit()

In [3]:
# # Get the source code of the html with requests
# # This method can NOT retrieve the following pages of dynamic webpage
# result = ""
# while result == "":
#     try:
#         # Certificate is not verified to bypass the SSLError
#         # Not secure though
#         result = requests.get(url, verify = False)
#         break
#     except:
#         sleep(5)
#         continue
# # Transform it into an element tree
# root = etree.fromstring(result.content, etree.HTMLParser())
# # Parse the information with Xpath
# # The Xpath for root is somehow different
# for row in root.xpath("//table[@class='ms-listviewtable']/tr"):
#     tests["test_ename"].append(row.xpath("./td/div/a/text()")[0].strip())
#     tests["test_cname"].append(row.xpath("./td/text()")[0].strip())
#     try:
#         tests["specimen"].append(row.xpath("./td/div/font/text()")[0].strip())
#     except IndexError:
#         tests["specimen"].append(row.xpath("./td/div/div/text()")[0].strip())
#     tests["reference_range"].append("\n".join(map(lambda x: x.strip(),row.xpath("./td/div/font/text()")[1:])))
#     tests["detail_link"].append(row.xpath("./td/div/a/@href")[0].strip())

In [4]:
pd.DataFrame(tests)[["test_ename", "test_cname", "specimen", "reference_range", "detail_link"]]

,test_ename,test_cname,specimen,reference_range,detail_link
0,(Iron & TIBC)_Iron,血清鐵,血清(Serum),51~209 µg/dL (2013/11/13變更)\n(2012/07/05～2013/...,https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
1,(Iron & TIBC)_TIBC,總鐵結合能力,血清(Serum),268~593 µg/dL (2013/11/13變更，舊參考值為 275 - 332 µg...,https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
2,17-KS (Urine),尿液17-酮類固醇定量,二十四小時尿液(24hrs Urine),♂：6 - 22 mg/24hrs\n♀：4 - 16 mg/24hrs,https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
3,25-OH Vitamin D,維生素D,血清(Serum),,https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
4,Acetaminophen(STAT),鎮痛解熱藥物Acetaminophen濃度,血清(Serum),有效治療濃度範圍：10 ~ 30 µg/mL,https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
5,"Adenosine Deaminase, ADA",腺核苷去胺酶,血清(Serum),,https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
6,"Adenosine Deaminase, ADA_CSF,PLE",腺核苷去胺酶,胸水(Pleural effusion)；腦脊髓液(CSF),,https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
7,"Adrenocorticotropic hormone,ACTH",腎上腺皮質激素,血漿(EDTA Plasma),7.40 ~ 57.30 pg/mL (2013/11/13變更，舊參考值為 10 ~ 65...,https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
8,AFP＋β-HCG＋uE3＋Inhibin A,懷孕第二期唐氏症篩檢（試劑檢驗）,血清(Serum),不適用(N/A),https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
9,Albumin(Blood),白蛋白,血清(Serum),3.5~5.7 g/dL (2013/11/13變更，舊參考值為3.5~5.0 g/dL),https://www.ntuh.gov.tw/labmed/檢驗目錄/_layouts/1...
